In [1]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import tensorflow

In [2]:
df=pd.read_csv(r'D:\Datasets\real_or_not.csv')

In [3]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df=df.drop(columns=['keyword','location'])

In [16]:
df.shape

(7613, 3)

In [5]:
df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
df.isna().sum()

id        0
text      0
target    0
dtype: int64

In [8]:
## Text pre_processing
x=df['text']
y=df['target']

In [9]:
import re
from nltk.corpus import stopwords

In [12]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(x)):
    
    review = re.sub('[^a-zA-Z0-9]', ' ', df['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [13]:
corpus

['deed reason earthquak may allah forgiv us',
 'forest fire near la rong sask canada',
 'resid ask shelter place notifi offic evacu shelter place order expect',
 '13 000 peopl receiv wildfir evacu order california',
 'got sent photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi 20 close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manit colorado spring area',
 'top hill see fire wood',
 'emerg evacu happen build across street',
 'afraid tornado come area',
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 'rain flood florida tampabay tampa 18 19 day lost count',
 'flood bago myanmar arriv bago',
 'damag school bu 80 multi car crash break',
 'man',
 'love fruit',
 'summer love',
 'car fast',
 'goooooooaaaaaal',
 'ridicul',
 'london cool',
 'love ski',
 'wonder day',
 'looooool',
 'way eat shit',
 'nyc last week',
 'love girlfriend',
 'cooool'

In [27]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [17]:
### Vocabulary size
voc_size=8000

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[5879, 7889, 4085, 87, 6628, 5129, 3095],
 [6950, 5130, 1350, 1939, 3532, 5934, 1745],
 [911, 7743, 1785, 4988, 3904, 7503, 3357, 1785, 4988, 4156, 232],
 [2998, 4352, 5935, 2327, 4902, 3357, 4156, 3694],
 [3181, 1578, 1615, 6829, 7576, 3488, 4902, 1912, 4289],
 [4617, 1087, 3694, 1410, 754, 2023, 4639, 4679, 1068, 7486, 5130, 2331, 4902],
 [3015, 1862, 3650, 2156, 5215, 6375, 3015, 4448, 528, 7046, 5295, 3238],
 [7600, 3570, 6499, 5130, 3655],
 [7981, 3357, 3047, 2793, 1077, 4448],
 [6045, 4713, 7441, 3238],
 [7277, 5935, 2637, 3387, 5156, 2404],
 [3764,
  4788,
  2040,
  1632,
  3015,
  4012,
  3630,
  2799,
  5835,
  4788,
  2040,
  2105,
  2105,
  138,
  3015],
 [2156, 3015, 2356, 5014, 2040, 345, 2063, 7378, 4979, 4659],
 [3015, 3028, 4420, 2660, 3028],
 [4748, 4289, 449, 4246, 7868, 7748, 3011, 4982],
 [4452],
 [5115, 3664],
 [3851, 5115],
 [7748, 2854],
 [2804],
 [3338],
 [5632, 4040],
 [5115, 1181],
 [7053, 7378],
 [3164],
 [698, 7098, 6042],
 [4859, 3071, 3486],
 [5115, 6342]

In [20]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[5879 7889 4085 ...    0    0    0]
 [6950 5130 1350 ...    0    0    0]
 [ 911 7743 1785 ...    0    0    0]
 ...
 [1978 2240 4708 ...    0    0    0]
 [5704 7615 6926 ...    0    0    0]
 [5509  621   23 ...    0    0    0]]


In [40]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 25, 40)            320000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 376,501
Trainable params: 376,501
Non-trainable params: 0
_________________________________________________________________


In [29]:
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [51]:
x_final,y_final

(array([[5879, 7889, 4085, ...,    0,    0,    0],
        [6950, 5130, 1350, ...,    0,    0,    0],
        [ 911, 7743, 1785, ...,    0,    0,    0],
        ...,
        [1978, 2240, 4708, ...,    0,    0,    0],
        [5704, 7615, 6926, ...,    0,    0,    0],
        [5509,  621,   23, ...,    0,    0,    0]]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [31]:
model.fit(X_final,y_final,epochs=10,batch_size=64)

Epoch 1/10
119/119 [==============================] - 5s 30ms/step - loss: 0.5547 - accuracy: 0.7157
Epoch 2/10
119/119 [==============================] - 3s 26ms/step - loss: 0.3629 - accuracy: 0.8530
Epoch 3/10
119/119 [==============================] - 3s 27ms/step - loss: 0.2741 - accuracy: 0.8990
Epoch 4/10
119/119 [==============================] - 3s 25ms/step - loss: 0.2160 - accuracy: 0.9267
Epoch 5/10
119/119 [==============================] - 3s 26ms/step - loss: 0.1731 - accuracy: 0.9418
Epoch 6/10
119/119 [==============================] - 3s 25ms/step - loss: 0.1450 - accuracy: 0.9517
Epoch 7/10
119/119 [==============================] - 3s 28ms/step - loss: 0.1211 - accuracy: 0.9631
Epoch 8/10
119/119 [==============================] - 3s 26ms/step - loss: 0.1086 - accuracy: 0.9693
Epoch 9/10
119/119 [==============================] - 3s 26ms/step - loss: 0.0962 - accuracy: 0.9735
Epoch 10/10
119/119 [==============================] - 3s 25ms/step - loss: 0.0796 - accura

In [32]:
# with bidirectional lstm

In [33]:
from tensorflow.keras.layers import Bidirectional

In [37]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [43]:
x_final1=np.array(embedded_docs)
y_final1=np.array(y)

In [45]:
model1.fit(X_final,y_final,epochs=10,batch_size=64)

Epoch 1/10
119/119 [==============================] - 3s 28ms/step - loss: 0.0316 - accuracy: 0.9900
Epoch 2/10
119/119 [==============================] - 3s 28ms/step - loss: 0.0314 - accuracy: 0.9899
Epoch 3/10
119/119 [==============================] - 3s 28ms/step - loss: 0.0312 - accuracy: 0.9888
Epoch 4/10
119/119 [==============================] - 4s 30ms/step - loss: 0.0257 - accuracy: 0.9912
Epoch 5/10
119/119 [==============================] - 4s 32ms/step - loss: 0.0252 - accuracy: 0.9909
Epoch 6/10
119/119 [==============================] - 3s 28ms/step - loss: 0.0264 - accuracy: 0.9903
Epoch 7/10
119/119 [==============================] - 3s 28ms/step - loss: 0.0230 - accuracy: 0.9916
Epoch 8/10
119/119 [==============================] - 4s 31ms/step - loss: 0.0203 - accuracy: 0.9929
Epoch 9/10
119/119 [==============================] - 4s 30ms/step - loss: 0.0190 - accuracy: 0.9934
Epoch 10/10
119/119 [==============================] - 4s 35ms/step - loss: 0.0159 - accura

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [47]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
80/80 [==============================] - 4s 45ms/step - loss: 0.0140 - accuracy: 0.9947 - val_loss: 0.0190 - val_accuracy: 0.9940
Epoch 2/10
80/80 [==============================] - 3s 34ms/step - loss: 0.0111 - accuracy: 0.9953 - val_loss: 0.0207 - val_accuracy: 0.9940
Epoch 3/10
80/80 [==============================] - 3s 40ms/step - loss: 0.0125 - accuracy: 0.9949 - val_loss: 0.0243 - val_accuracy: 0.9924
Epoch 4/10
80/80 [==============================] - 3s 38ms/step - loss: 0.0116 - accuracy: 0.9951 - val_loss: 0.0242 - val_accuracy: 0.9928
Epoch 5/10
80/80 [==============================] - 3s 39ms/step - loss: 0.0101 - accuracy: 0.9965 - val_loss: 0.0265 - val_accuracy: 0.9924
Epoch 6/10
80/80 [==============================] - 4s 45ms/step - loss: 0.0099 - accuracy: 0.9961 - val_loss: 0.0326 - val_accuracy: 0.9889
Epoch 7/10
80/80 [==============================] - 4s 45ms/step - loss: 0.0102 - accuracy: 0.9957 - val_loss: 0.0358 - val_accuracy: 0.9889
Epoch 8/10
80

In [49]:
y_pred1=model1.predict(X_test)

In [53]:
sklearn.metrics.r2_score¶
print(confusion_matrix(y_test,y_pred1))

ValueError: Classification metrics can't handle a mix of binary and continuous targets